**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

C:\Users\Ahmed\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act returns the action that the agent perform given a state. It will perform a random action with a probability epsilon, or the learned action else.

Epsilon is essential because it allows the algorithm to explore the Action space. This is called 'exploration'. If epsilon is not used during the training, the algorithm can easily converge to a suboptimal solution and can't ameliore it if it don't explore other possibilities. 

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=51 # set small when debugging
epochs_test=11 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

**Answer**

The position array gives the position the rat at a given moment. 
The board array gives the position of the poisonous cells and the cheese.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.choice(4)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state , train = False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
        
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 6.5/12.0. Average score (-5.5)
Win/lose count 6.5/17.0. Average score (-8.0)
Win/lose count 10.0/17.0. Average score (-7.666666666666667)
Win/lose count 6.0/16.0. Average score (-8.25)
Win/lose count 13.5/8.0. Average score (-5.5)
Win/lose count 6.5/14.0. Average score (-5.833333333333333)
Win/lose count 10.5/13.0. Average score (-5.357142857142857)
Win/lose count 10.0/14.0. Average score (-5.1875)
Win/lose count 8.0/13.0. Average score (-5.166666666666667)
Win/lose count 10.5/19.0. Average score (-5.5)
Win/lose count 4.0/17.0. Average score (-6.181818181818182)
Final score: -6.181818181818182


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




**Answer:**

**1-** We have that:

$$ Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> $$

thus:

$$ Q^\pi(s,a)=E_{p^{\pi}}[ r(s_{t},a_{t}) + \sum_{1 \leq t}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> $$

$$ Q^\pi(s,a)=E_{p^{\pi}}[ r(s,a) + \gamma \sum_{1 \leq t}\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> $$

thus (with $ (s_{t},a_{t}) = (s_{t}^\prime,a_{t}^\prime) $ and $ (s^\prime,a^\prime) = (s_1 , a_1)$  )

$$ Q^\pi(s,a)=E_{(s',a')\sim p(.|s,a)}[ r(s,a) + E_{p^{\pi}}[ \gamma \sum_{t}\gamma^{t}r(s_{t}^\prime,a_{t}^\prime)|s_{0}^\prime=s^\prime,a_{0}^\prime=a^\prime] ] \> $$

and finally:

$$ Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] $$


**2-**

We have the following relationship between the optimal policy and the optimal Q function:

(1) : $$ \pi^*(s) = \max_a Q^*(s,a) $$

With the previous equation:

$$ Q^{*}(s,a)=E_{(s',a')\sim \pi^* (.|s,a)}[r(s,a)+\gamma Q^{\pi^*}(s',\pi^*(s'))] $$

By replacing $ \pi^*(s) $ by its expression in (1), We have :

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}

** 3- **

The previous equation show that: 

\begin{equation*}
E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q^*(s',a',\theta)-Q^*(s,a,\theta)\Vert^{2} = 0 \text{ if Q = Q* and is strictly positive otherwise}
\end{equation*}

Thus minimizing the following objective is a viable option for training the DQN:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        self.memory = self.memory[-self.max_memory:]

    def random_access(self):
        return self.memory[np.random.choice(len(self.memory))]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
from keras.layers.core import Dense, Flatten

class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape(1 , 5 , 5 , self.n_state)))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            input_states[i] = s_
            target_q[i] = self.model.predict(s_.reshape(1 , 5 , 5 , self.n_state))
            if game_over_:
                ######## FILL IN
                target_q[i,a_] = r_
            else:
                ######## FILL IN
                target_q[i,a_] = r_ + self.discount*np.max(self.model.predict(n_s_.reshape(1 , 5 , 5 , self.n_state)))
                
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -2, 2)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        model = Sequential()
        model.add(Flatten(input_shape = (5 , 5 , self.n_state)))
        model.add(Dense(64))
        model.add(Activation('relu'))
        model.add(Dense(64))
        model.add(Activation('relu'))
        model.add(Dense(4))
        
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train50.mp4'))

Epoch 000/051 | Loss 0.0025 | Win/lose count 1.0/4.0 (-3.0)
Epoch 001/051 | Loss 0.0025 | Win/lose count 4.0/5.0 (-1.0)
Epoch 002/051 | Loss 0.0064 | Win/lose count 3.5/4.0 (-0.5)
Epoch 003/051 | Loss 0.0025 | Win/lose count 4.5/10.0 (-5.5)
Epoch 004/051 | Loss 0.0104 | Win/lose count 4.0/3.0 (1.0)
Epoch 005/051 | Loss 0.0112 | Win/lose count 8.0/3.0 (5.0)
Epoch 006/051 | Loss 0.0044 | Win/lose count 1.5/2.0 (-0.5)
Epoch 007/051 | Loss 0.0115 | Win/lose count 4.0/4.0 (0.0)
Epoch 008/051 | Loss 0.0080 | Win/lose count 2.5/4.0 (-1.5)
Epoch 009/051 | Loss 0.0019 | Win/lose count 0/2.0 (-2.0)
Epoch 010/051 | Loss 0.0072 | Win/lose count 9.0/8.0 (1.0)
Epoch 011/051 | Loss 0.0032 | Win/lose count 4.0/2.0 (2.0)
Epoch 012/051 | Loss 0.0083 | Win/lose count 7.0/4.0 (3.0)
Epoch 013/051 | Loss 0.0122 | Win/lose count 6.5/3.0 (3.5)
Epoch 014/051 | Loss 0.0021 | Win/lose count 4.5/3.0 (1.5)
Epoch 015/051 | Loss 0.0218 | Win/lose count 8.0/7.0 (1.0)
Epoch 016/051 | Loss 0.0028 | Win/lose count 6.5/2

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [13]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        
        model.add(Conv2D(32 , 2 , input_shape = (5,5,self.n_state) , padding= 'same'))
        model.add(Activation('relu'))
        model.add(Conv2D(64 , 2))
        model.add(Activation('relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train50.mp4'))

Epoch 000/051 | Loss 0.0012 | Win/lose count 3.0/0 (3.0)
Epoch 001/051 | Loss 0.0006 | Win/lose count 3.5/7.0 (-3.5)
Epoch 002/051 | Loss 0.0026 | Win/lose count 3.0/0 (3.0)
Epoch 003/051 | Loss 0.0280 | Win/lose count 3.5/1.0 (2.5)
Epoch 004/051 | Loss 0.0071 | Win/lose count 3.5/5.0 (-1.5)
Epoch 005/051 | Loss 0.0006 | Win/lose count 3.0/3.0 (0.0)
Epoch 006/051 | Loss 0.0040 | Win/lose count 0.5/4.0 (-3.5)
Epoch 007/051 | Loss 0.0058 | Win/lose count 3.5/2.0 (1.5)
Epoch 008/051 | Loss 0.0010 | Win/lose count 4.0/2.0 (2.0)
Epoch 009/051 | Loss 0.0020 | Win/lose count 2.5/2.0 (0.5)
Epoch 010/051 | Loss 0.0218 | Win/lose count 3.5/0 (3.5)
Epoch 011/051 | Loss 0.0060 | Win/lose count 4.5/5.0 (-0.5)
Epoch 012/051 | Loss 0.0016 | Win/lose count 3.0/2.0 (1.0)
Epoch 013/051 | Loss 0.0048 | Win/lose count 5.0/1.0 (4.0)
Epoch 014/051 | Loss 0.0032 | Win/lose count 5.5/1.0 (4.5)
Epoch 015/051 | Loss 0.0006 | Win/lose count 3.5/4.0 (-0.5)
Epoch 016/051 | Loss 0.0052 | Win/lose count 6.0/1.0 (5.0

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [24]:
env = Environment(grid_size=size, max_time=T,temperature=0.1)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

HTML(display_videos('cnn_test1.mp4'))

Test of the CNN
Win/lose count 0.5/0. Average score (0.5)
Win/lose count 2.0/0. Average score (1.25)
Win/lose count 0.5/0. Average score (1.0)
Win/lose count 0/0. Average score (0.75)
Win/lose count 0/0. Average score (0.6)
Win/lose count 2.0/0. Average score (0.8333333333333334)
Win/lose count 1.5/0. Average score (0.9285714285714286)
Win/lose count 0/0. Average score (0.8125)
Win/lose count 2.5/0. Average score (1.0)
Win/lose count 0/0. Average score (0.9)
Win/lose count 0.5/0. Average score (0.8636363636363636)
Final score: 0.8636363636363636
Test of the FC
Win/lose count 0/1.0. Average score (-1.0)
Win/lose count 0/1.0. Average score (-1.0)
Win/lose count 0/0. Average score (-0.6666666666666666)
Win/lose count 0/1.0. Average score (-0.75)
Win/lose count 0.5/0. Average score (-0.5)
Win/lose count 0/1.0. Average score (-0.5833333333333334)
Win/lose count 0/1.0. Average score (-0.6428571428571429)
Win/lose count 0.5/2.0. Average score (-0.75)
Win/lose count 1.0/0. Average score (-0.55

In [25]:
HTML(display_videos('fc_test1.mp4'))

In [27]:
env = Environment(grid_size=size, max_time=T,temperature=0.8)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

HTML(display_videos('cnn_test1.mp4'))


Test of the CNN
Win/lose count 1.5/0. Average score (1.5)
Win/lose count 17.0/1.0. Average score (8.75)
Win/lose count 3.0/0. Average score (6.833333333333333)
Win/lose count 8.5/0. Average score (7.25)
Win/lose count 1.0/0. Average score (6.0)
Win/lose count 19.0/0. Average score (8.166666666666666)
Win/lose count 17.0/0. Average score (9.428571428571429)
Win/lose count 3.0/0. Average score (8.625)
Win/lose count 10.0/0. Average score (8.777777777777779)
Win/lose count 4.0/0. Average score (8.3)
Win/lose count 5.5/0. Average score (8.045454545454545)
Final score: 8.045454545454545
Test of the FC
Win/lose count 4.5/3.0. Average score (1.5)
Win/lose count 0.5/0. Average score (1.0)
Win/lose count 3.0/5.0. Average score (0.0)
Win/lose count 2.5/2.0. Average score (0.125)
Win/lose count 0/1.0. Average score (-0.1)
Win/lose count 7.0/2.0. Average score (0.75)
Win/lose count 3.5/0. Average score (1.1428571428571428)
Win/lose count 1.5/1.0. Average score (1.0625)
Win/lose count 1.5/2.0. Aver

In [26]:
HTML(display_videos('fc_test1.mp4'))

In [30]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

HTML(display_videos('cnn_test3.mp4'))


Test of the CNN
Win/lose count 2.5/0. Average score (2.5)
Win/lose count 3.5/0. Average score (3.0)
Win/lose count 2.5/0. Average score (2.8333333333333335)
Win/lose count 1.0/0. Average score (2.375)
Win/lose count 1.5/0. Average score (2.2)
Win/lose count 0/0. Average score (1.8333333333333333)
Win/lose count 0.5/0. Average score (1.6428571428571428)
Win/lose count 0.5/0. Average score (1.5)
Win/lose count 7.5/0. Average score (2.1666666666666665)
Win/lose count 3.0/0. Average score (2.25)
Win/lose count 0/0. Average score (2.0454545454545454)
Final score: 2.0454545454545454
Test of the FC
Win/lose count 0.5/0. Average score (0.5)
Win/lose count 0.5/2.0. Average score (-0.5)
Win/lose count 1.0/1.0. Average score (-0.3333333333333333)
Win/lose count 0/0. Average score (-0.25)
Win/lose count 0.5/0. Average score (-0.1)
Win/lose count 1.0/0. Average score (0.08333333333333333)
Win/lose count 0.5/1.0. Average score (0.0)
Win/lose count 1.0/1.0. Average score (0.0)
Win/lose count 1.5/0. A

In [29]:
HTML(display_videos('fc_test1.mp4'))

**Answer**

The CNN performs better in average than the Fully Connected layer. But we observe that the FC agent and the CNN agent both stay in a confined region of the space and do not explore the totality of the board. This is an issue because it means that the agents did not converge to optimal solutions.

However when the temperature is higher, the agent tends to move more freely due to a higher expected value function. 

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [18]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    eps_start , eps_decay = 0.8 , 0.9

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        #### Exploraton probability
        agent.set_epsilon(eps_start * (eps_decay**e) )

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action , train = True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
        

        
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        super(EnvironmentExploring, self).__init__(grid_size, max_time, temperature)
    
    def act(self, action , train = True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        
        # Exploration
        if train:
            reward = reward - self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        
        
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.malus_position = np.zeros((self.grid_size,self.grid_size))
        
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        
        self.board[self.x,self.y] = 0
        self.malus_position[self.x, self.y] = 0.1
        
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    
    

##########################################    

# ## use those samples of code:
# #In train explore:
# state, reward, game_over = env.act(action, train=True)

# ## In Environment exploring:
# # You will have to change n_state to 3 because you will use one more layer!
# reward = 0
# if train:
#     reward = -self.malus_position[self.x, self.y]
# self.malus_position[self.x, self.y] = 0.1

# reward = reward + self.board[self.x, self.y]
# # 3 "feature" states instead of 2
# state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
#                                 self.board.reshape(self.grid_size, self.grid_size,1),
#                         self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

In [19]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/051 | Loss 0.0119 | Win/lose count 10.5/30.500000000000107 (-20.000000000000107)
Epoch 001/051 | Loss 0.0106 | Win/lose count 9.0/19.700000000000006 (-10.700000000000006)
Epoch 002/051 | Loss 0.0029 | Win/lose count 12.0/24.900000000000077 (-12.900000000000077)
Epoch 003/051 | Loss 0.0034 | Win/lose count 14.5/22.099999999999998 (-7.599999999999998)
Epoch 004/051 | Loss 0.0042 | Win/lose count 14.0/22.800000000000043 (-8.800000000000043)
Epoch 005/051 | Loss 0.0289 | Win/lose count 17.5/20.600000000000026 (-3.1000000000000263)
Epoch 006/051 | Loss 0.0104 | Win/lose count 10.0/19.39999999999999 (-9.399999999999991)
Epoch 007/051 | Loss 0.0071 | Win/lose count 14.5/21.100000000000037 (-6.600000000000037)
Epoch 008/051 | Loss 0.0237 | Win/lose count 18.5/22.100000000000023 (-3.6000000000000227)
Epoch 009/051 | Loss 0.0020 | Win/lose count 16.5/19.299999999999986 (-2.7999999999999865)
Epoch 010/051 | Loss 0.0060 | Win/lose count 22.5/17.79999999999999 (4.70000000000001)
Epoch 011

In [20]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 7.5/17.999999999999986. Average score (-10.499999999999986)
Win/lose count 12.0/15.199999999999962. Average score (-6.849999999999974)
Win/lose count 12.5/14.599999999999964. Average score (-5.266666666666637)
Win/lose count 14.5/15.59999999999996. Average score (-4.224999999999968)
Win/lose count 16.5/12.499999999999972. Average score (-2.5799999999999685)
Win/lose count 14.0/14.499999999999964. Average score (-2.233333333333301)
Win/lose count 15.5/16.699999999999967. Average score (-2.0857142857142534)
Win/lose count 18.0/12.599999999999971. Average score (-1.1499999999999682)
Win/lose count 20.0/15.49999999999996. Average score (-0.5222222222221894)
Win/lose count 9.0/16.19999999999996. Average score (-1.1899999999999664)
Win/lose count 4.5/19.300000000000008. Average score (-2.4272727272726975)
Final score: -2.4272727272726975


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In [21]:
def train_other(agent1 , agent2 ,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    eps_start , eps_decay = 0.8 , 0.9

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent2.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action , train = True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent1.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent1.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [22]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent_m = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_other(agent_m , agent, env, epochs_train, prefix='cnn_train_other')
HTML(display_videos('cnn_train_other10.mp4'))

Epoch 000/051 | Loss 0.0010 | Win/lose count 25.0/14.299999999999965 (10.700000000000035)
Epoch 001/051 | Loss 0.0025 | Win/lose count 21.5/12.499999999999972 (9.000000000000028)
Epoch 002/051 | Loss 0.0007 | Win/lose count 12.0/15.099999999999962 (-3.0999999999999623)
Epoch 003/051 | Loss 0.0003 | Win/lose count 11.5/14.399999999999965 (-2.899999999999965)
Epoch 004/051 | Loss 0.0304 | Win/lose count 3.0/19.0 (-16.0)
Epoch 005/051 | Loss 0.0288 | Win/lose count 22.5/12.099999999999973 (10.400000000000027)
Epoch 006/051 | Loss 0.0288 | Win/lose count 14.0/14.499999999999964 (-0.4999999999999645)
Epoch 007/051 | Loss 0.0008 | Win/lose count 21.0/14.599999999999964 (6.400000000000036)
Epoch 008/051 | Loss 0.0012 | Win/lose count 8.5/17.59999999999998 (-9.09999999999998)
Epoch 009/051 | Loss 0.0280 | Win/lose count 12.5/15.099999999999964 (-2.599999999999964)
Epoch 010/051 | Loss 0.0004 | Win/lose count 23.0/13.999999999999966 (9.000000000000034)
Epoch 011/051 | Loss 0.0008 | Win/lose cou

In [23]:
test(agent_m,env,epochs_test,prefix='cnn_test_other')
HTML(display_videos('cnn_test_other9.mp4'))

Win/lose count 6.0/18.19999999999999. Average score (-12.199999999999989)
Win/lose count 8.5/16.79999999999997. Average score (-10.249999999999979)
Win/lose count 8.0/18.39999999999999. Average score (-10.299999999999983)
Win/lose count 5.5/18.19999999999999. Average score (-10.899999999999984)
Win/lose count 8.5/17.399999999999977. Average score (-10.499999999999982)
Win/lose count 7.0/17.899999999999984. Average score (-10.56666666666665)
Win/lose count 8.5/17.599999999999984. Average score (-10.357142857142842)
Win/lose count 1.0/20.600000000000026. Average score (-11.512499999999989)
Win/lose count 4.5/18.699999999999996. Average score (-11.8111111111111)
Win/lose count 8.0/18.799999999999997. Average score (-11.70999999999999)
Win/lose count 7.5/18.099999999999987. Average score (-11.609090909090897)
Final score: -11.609090909090897


We can see that the agent can't perform better than the original agent.

***